In [4]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from typing import List, Dict, Any
import threading
from queue import Queue
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from dataclasses import dataclass
from datetime import datetime
import logging
import os
from threading import Lock
import numpy as np
import asyncio
import aiohttp
import uvicorn
from fastapi import FastAPI, HTTPException, BackgroundTasks
from pydantic import BaseModel

In [ ]:
@dataclass
class PredictionRequest:
    message_id: str
    user_id: str
    text: str
    timestamp: datetime

    def __post_init__(self):
        if self.timestamp is None:
            self.timestamp = time.time()

In [ ]:
@dataclass
class PredictionResult:
    message_id: str
    user_id: str
    is_bullying: bool
    confidence: float
    probability_bullying: float
    # processing_time: float
    # model_version: str

In [ ]:
class SingleTextRequest(BaseModel):
    text: str
    user_id: str = "unknown"
    message_id: Optional[str] = None

class BatchTextRequest(BaseModel):
    messages: List[Dict[str, str]]


In [ ]:
class PredictionResponse(BaseModel):
    message_id: str
    user_id: str
    is_bullying: bool
    confidence: float
    probability_bullying: float
    processing_time: float
    model_version: str
    alert_level: str
    timestamp: str

class BatchPredictionResponse(BaseModel):
    results: List[PredictionResponse]
    total_processed: int
    total_processing_time: float
    average_processing_time: float


In [ ]:
class HealthResponse(BaseModel):
    status: str
    model_loaded: bool
    stats: Dict
    uptime: str


In [ ]:
class CyberbullyingDetector:
    def __init__(self, model_path: str, host: str = "0.0.0.0", port: int = 8080, **kwargs):
        self.model_path = model_path
        self.host = host
        self.port = port
        self.start_time = time.time()
        
        # Initialize the cyberbullying API
        logger.info("Loading cyberbullying model...")
        self.api = CyberbullyingAPI(model_path, **kwargs)
        logger.info("Model loaded successfully!")
        

        self.app = FastAPI(
            title="Cyberbullying Detection API",
            description="Real-time cyberbullying detection service",
            version="1.0.0"
        )

        